<a href="https://colab.research.google.com/github/mechhector/leaf_recognition_paper_ICPR2022/blob/main/leaf_recognition_paper_ICPR2022/classifiers_leaf_recognition_paper_ICPR22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import time
import os

from scipy import stats
from scipy import ndimage

from skimage.feature import greycomatrix, greycoprops, local_binary_pattern
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_score, StratifiedShuffleSplit
from sklearn.feature_extraction import image
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from google.colab.patches import cv2_imshow

dataset = 'FLAVIA'

In [ ]:
# Dataset shape and texture features

shape_features_file = np.loadtxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_dataset_features/{}_shape_features.txt'.format(dataset,dataset), delimiter = ',')

texture_features_file = np.loadtxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_dataset_features/{}_texture_features.txt'.format(dataset,dataset), delimiter = ',')

glcm_features_file = np.loadtxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_dataset_features/{}_glcm_features.txt'.format(dataset,dataset), delimiter = ',')

shape_glcm_features_file = np.loadtxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_dataset_features/{}_shape_glcm_features.txt'.format(dataset,dataset), delimiter =',' )

shape_texture_features_file = np.loadtxt('/content/drive/MyDrive/leaf_classification_paper/data/leaf_rec_paper/{}_dataset_features/{}_shape_texture_features.txt'.format(dataset,dataset), delimiter = ',')

In [ ]:
shape_features_file.shape

(1201, 8)

In [ ]:
shape_features = shape_features_file[:,2:]

texture_features = texture_features_file

glcm_features = glcm_features_file

shape_glcm_features = shape_glcm_features_file[:,2:]

shape_texture_features = shape_texture_features_file[:,2:]


features_files_list = [shape_features, texture_features, glcm_features, shape_glcm_features, shape_texture_features]

features_files_list_str = ['shape_features', 'texture_features', 'glcm_features', 'shape_glcm_features','shape_texture_features']

for ki in features_files_list:
  print(ki.shape)


(1201, 6)
(1201, 5)
(1201, 5)
(1201, 11)
(1201, 11)


In [ ]:
targets_vector = shape_features_file[ : , 0]
#print(targets_vector)

In [ ]:
start_time = time.time()
#main()

data_list = [shape_features, shape_glcm_features, shape_texture_features]
data_name =  ['shape_features', 'shape_glcm_features','shape_texture_features']

hd = 0 

#clf = KNeighborsClassifier(n_neighbors=1)
#clf = MLPClassifier(random_state=1, max_iter=10000)
#clf = tree.DecisionTreeClassifier()
#clf = GaussianNB()
clf = make_pipeline(StandardScaler(), SVC(kernel = 'linear', gamma='scale'))

print(dataset, "Number of samples:", shape_features_file.shape[0])

for p in data_list: 

  X = p
  y = np.asarray(targets_vector)
  

# 10-fold cross validation

  sss = StratifiedShuffleSplit(n_splits=10 , test_size=0.2)

  for train_index , test_index in sss.split(X,y): 
    X_train , X_test = X[train_index] , X[test_index]
    y_train , y_test = y[train_index] , y[test_index]



  # Acc, std deviation and confusion matrix

  std = []

  mean_acc = 0

  c_matrix = 0

  for k, (train, test) in enumerate(sss.split(X, y)):
      clf.fit(X[train], y[train])

      predicted = clf.predict(X_test)

      acc = metrics.accuracy_score(y_test, predicted)

      std.append(acc)

      mean_acc = mean_acc + acc

      c_matrix = c_matrix + confusion_matrix(y_test, predicted)

      #print(confusion_matrix(y_test, predicted))
      #print(confusion_matrix(y_test, predicted).sum())

      #print("Accuracy: " f"{metrics.accuracy_score(y_test, predicted)}\n")

  mean_c_matrix = c_matrix/10
  for i in range(mean_c_matrix.shape[0]):
    for j in range(mean_c_matrix.shape[1]):
      c_matrix[i][j] = round(c_matrix[i][j])    

  #print('mean confusion matrix:') 
  #print(c_matrix/10) 

  
  print(data_name[hd], str(clf))   
 
  print('mean accuracy: ' , mean_acc/10) 

  #print('test samples: ' , c_matrix.sum()/10)

  print('std deviation: ' , np.asarray(std).std())
  print('\n')
  

  hd = hd + 1



print("--- %s seconds ---" % (time.time() - start_time))

'''
# To plot the confusion matrix

plt.figure(figsize=(11,7))
sn.set(font_scale=1.8) 
sn.heatmap(c_matrix/10, annot=True, annot_kws={"size": 18}) # font size
plt.suptitle("Matriz de Confusão")
plt.xlabel("Predito")
plt.ylabel("Real")
plt.gcf().set_dpi(50)
plt.show()
'''

FLAVIA Number of samples: 1201
shape_features Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])
mean accuracy:  0.6016597510373443
std deviation:  0.00927829036306967


shape_glcm_features Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])
mean accuracy:  0.6813278008298757
std deviation:  0.008257157154411755


shape_texture_features Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])
mean accuracy:  0.7634854771784232
std deviation:  0.009090830829961271


--- 1.4624829292297363 seconds ---


'\nplt.figure(figsize=(11,7))\nsn.set(font_scale=1.8) \nsn.heatmap(c_matrix/10, annot=True, annot_kws={"size": 18}) # font size\nplt.suptitle("Matriz de Confusão")\nplt.xlabel("Predito")\nplt.ylabel("Real")\nplt.gcf().set_dpi(50)\nplt.show()\n'